In [3]:
# 探究流水线拒绝回答无关问题的能力
import utils
from query import QueryManager
from chat import ChatManager
from advance import ReformatManager, RejectManager, HistoryManager
from knowledgeGraph.chatbot_graph import ChatBotGraph

import warnings    

In [5]:
warnings.filterwarnings('ignore')
utils.setup()

print("正在加载知识图谱...")
graphChat = ChatBotGraph()

print("正在加载数据库...")
queryManager = QueryManager()

def test(name, questions):
    rejectManager = RejectManager()
    reformatManager = ReformatManager()
    historyManager = HistoryManager()
    chatManager = ChatManager()
    
    for question in questions:
        utils.Logger.log(f"./log/{name}.log", question)
        
        deter = rejectManager.determine(question)
        utils.Logger.log(f"./log/{name}.log", f"是否属于健康医疗领域：{deter}")
        
        if deter == "False":
            continue
        
        expand_question = historyManager.expand(question, chatManager.history)
        utils.Logger.log(f"./log/{name}.log", f"问句改写：{expand_question}")
        
        pairs = reformatManager.reformat(expand_question)
        utils.Logger.log(f"./log/{name}.log", f"问句拆分为{len(pairs)}个问题。")
        for idx, pair in enumerate(pairs):
            simplified_question, brief_ans = pair["question"], pair["answer"]
            utils.Logger.log(f"./log/{name}.log", f"问句{idx+1}：{simplified_question}\n回答{idx+1}: {brief_ans}")
            question = simplified_question
            query = f"{simplified_question} {brief_ans}"
            graph_ret = graphChat.chat_main(simplified_question)
            utils.Logger.log(f"./log/{name}.log", f"知识图谱结果：{graph_ret}")
            reference = queryManager.query_one(query)
            utils.Logger.log(f"./log/{name}.log", f"向量知识库结果：{reference}")
            
            if graph_ret is not None:
                reference = reference + f"\nknowledgeGraph: {graph_ret}"
            
            extract, response = chatManager.chat_one(question, reference)
            if extract == "":
                extract = "无"
            utils.Logger.log(f"./log/{name}.log", f"大模型结果：{response}\n参考知识：{extract}")


正在加载知识图谱...
正在加载数据库...


In [9]:
test_questions = {
    "reject": [
        "Python该怎么学？",
        "飞机有哪些部件组成？",
        "狂犬病疫苗去哪里接种？",
        "水杯一般多少钱？",
        "狂犬病疫苗打一次多少钱？",
        "没钱赚，好头疼",
        "今天吃什么？",
        "打狂犬病疫苗有什么后遗症？",
        "艾滋病的传染途径是什么？",
        "献血政策是什么?"
    ],
    "reject2": [
        "宠物陪伴对人的心理健康有何积极影响？",
        "空气污染如何影响儿童的学习能力？",
        "学习新语言能延缓衰老吗？",
        "为什么某些气味能唤起强烈的情感回忆？",
        "睡眠不足会影响记忆力吗？",
        "经常运动的人需要更多的蛋白质吗？",
        "冥想如何影响大脑的结构？",
        "饮用绿茶真的能帮助减肥吗？",
        "为什么在长时间阅读后眼睛会感到疲劳？",
        "为什么在飞机上耳朵会感到不适？"
    ]
}

In [10]:
for k, v in test_questions.items():
    test(k, v)